## Import libraries

In [9]:
%matplotlib qt

import os
import mne
import sys
import pyxdf
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.fft as fft
import scipy.stats as stats
import matplotlib.pyplot as plt
sys.path.append("..") # Adds higher directory to python modules path.
from Functions import eeg_quality_index

## Import data

In [10]:
# Get data directory
current_directory = os.getcwd()
os.chdir("..")
data_folder = os.getcwd() + '\\Data\\'
os.chdir(current_directory)

# - Create MNE object from XDF data
stream = 1  # Select proper stream from XDF file
trial_name = '\\220627 EZ Headset comparison\\EZ_gtec_baseline.xdf'  # Change this if needed
print(f'{data_folder+trial_name}')
streams, header = pyxdf.load_xdf(data_folder+trial_name, verbose=False)
sfreq = float(streams[stream]['info']['nominal_srate'][0])                  # Sampling frequency [Hz]
n_chans = len(streams[stream]['info']['desc'][0]['channels'][0]['channel']) # Number of channels [n]
chans = [streams[stream]['info']['desc'][0]['channels'][0]['channel'][i]['label'][0] for i in range(n_chans)]   # Channel names
info = mne.create_info(chans, sfreq, ch_types='eeg')
data = streams[stream]['time_series'].T # EEG data [uV]
gain = 1e-6 # Multiply data by gain factor to get the proper units
gtec = mne.io.RawArray(data*gain, info, verbose=False) # Change this variable name if needed

# - Import EDF data to MNE objects
# -- Cognionics
trial_name = '\\220627 EZ Headset comparison\\EZ_cgx_baseline.edf'
cgx = mne.io.read_raw_edf(data_folder+trial_name, verbose=False, preload=True)

# -- Wearable sensing
trial_name = '\\220627 EZ Headset comparison\\EZ_ws_baseline.edf'
ws = mne.io.read_raw_edf(data_folder+trial_name, verbose=False, preload=True)


c:\Users\danie\Documents\Projects\EEG_quality_index\Data\\220627 EZ Headset comparison\EZ_gtec_baseline.xdf


## Plot raw data

Enable if needed

In [11]:
# gtec.plot(title='Clean dataset')
# cgx.plot(title='Cognionics dataset')
# ws.plot(title='Wearable sensing dataset')

## Pick channels

From CGX and WS headsets, pick channels that are similar to the position of the Gtec headset

In [12]:
gtec_chans = ['FC3', 'FCz', 'FC4','C3', 'Cz', 'C4', 'P3', 'P4', 'PO7', 'PO8']
cgx_chans = ['F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'P4', 'O1', 'O2']
ws_chans = ['EEG F3-Pz', 'EEG Fz-Pz', 'EEG F4-Pz', 'EEG C3-Pz', 'EEG Cz-Pz', 'EEG C4-Pz',
            'EEG P3-Pz', 'EEG P4-Pz', 'EEG O1-Pz', 'EEG O2-Pz']

gtec.pick(gtec_chans, verbose=False)
cgx.pick(cgx_chans, verbose=False)
ws.pick(ws_chans, verbose=False)

<RawEDF | EZ_ws_baseline.edf, 10 x 46500 (155.0 s), ~3.6 MB, data loaded>

## Trim and filter data

In [13]:
#%% Filter data
# - Select cutoff frequencies for bandpass filter and filter the data
fc_high = 65    # cutoff frequency high [Hz]
fc_low = 0.1    # cutoff frequency low [Hz]

gtec.filter(l_freq=fc_low, h_freq=fc_high)
cgx.filter(l_freq=fc_low, h_freq=fc_high)
ws.filter(l_freq=fc_low, h_freq=fc_high)

#%% Trim times
trim_times = [45, 75]
gtec.crop(tmin=trim_times[0], tmax=trim_times[1])
cgx.crop(tmin=trim_times[0], tmax=trim_times[1])
ws.crop(tmin=trim_times[0], tmax=trim_times[1])


<RawEDF | EZ_ws_baseline.edf, 10 x 9001 (30.0 s), ~718 kB, data loaded>

## Compute EQI


In [14]:
# Numpy data
gtec_data = gtec.get_data()
cgx_data = cgx.get_data()
ws_data = ws.get_data()

# Sampling rates [Hz]
gtec_srate = int(gtec.info['sfreq'])
cgx_srate = int(cgx.info['sfreq'])
ws_srate = int(ws.info['sfreq'])

# Windows [samples]
gtec_win = int(gtec_srate/2)
cgx_win = int(cgx_srate/2)
ws_win = int(ws_srate/2)

# Sliding window [samples]
sliding_window = 10e-3  # Time of sliding window [sec]
gtec_slide = int(gtec_srate*sliding_window)
cgx_slide = int(cgx_srate*sliding_window)
ws_slide = int(ws_srate*sliding_window)

[_, cgx_percent, cgx_mean] = eeg_quality_index.scoring(gtec_data, cgx_data, gtec_srate, cgx_srate, window=[gtec_win, cgx_win], slide=[gtec_slide, cgx_slide])
[_, ws_percent, ws_mean] = eeg_quality_index.scoring(gtec_data, ws_data, gtec_srate, ws_srate, window=[gtec_win, ws_win], slide=[gtec_slide, ws_slide])

## Export results

In [17]:
results_cgx = pd.DataFrame(np.row_stack((cgx_percent, cgx_mean)), columns=cgx_chans)
results_ws = pd.DataFrame(np.row_stack((ws_percent, ws_mean)), columns=ws_chans)

excel_file = 'EZ_results_'
results_cgx.to_csv(excel_file+'cgx.csv')
results_ws.to_csv(excel_file+'ws.csv')